# Character Encodings

## 0. Modules

In [1]:
import numpy as np
import pandas as pd

# helpful character encoding module
import charset_normalizer

# set seed for reproducibility
np.random.seed(0)

## 1. What are encodings?

字元編碼是將原始二進制位元組串（看起來像這樣：0110100001101001）映射到構成人類可讀文本（如「hi」）的字元的特定規則集。  
有許多不同的編碼，如果你嘗試用與原始書寫不同的編碼來讀取文本，你最終會得到一種叫做「文字亂碼」（發音像 mo-gee-bah-kay）的混亂文本。  
以下是文字亂碼的一個例子：

æ–‡å—åŒ–ã??

你也可能遇到「未知」字符。當某個特定位元組與你用來讀取位元組串的編碼中的字符之間沒有映射時，  
就會顯示這些字符，它們看起來像這樣：

����������

字符編碼不匹配的問題今天比以前少見，但絕對仍然是一個問題。  
有許多不同的字符編碼，但你需要知道的主要編碼是 UTF-8。

**UTF-8 是標準文本編碼。**所有 Python 程式碼都使用 UTF-8，理想情況下，你的所有數據也應該如此。  
當事情不使用 UTF-8 時，你就會遇到麻煩。

在 Python 2 中處理編碼相當困難，但幸運的是在 Python 3 中這變得簡單許多。  
（Kaggle 筆記本僅使用 Python 3。）在 Python 3 中處理文本時，你會遇到兩種主要的數據類型。  
其中一種是字符串，這是文本的默認類型。

In [2]:
# start with a string
before = "This is the euro symbol: €"

# check to see what datatype it is
type(before)

str

另一種數據類型是位元組數據類型，它是一系列的整數。  
你可以通過指定其編碼來將字符串轉換成位元組：

In [3]:
# encode it to a different encoding, replacing characters that raise errors
after = before.encode("utf-8", errors="replace")

# check the type
type(after)

bytes

如果你觀察一個位元組對象，你會看到它前面有一個 b，然後可能在後面有一些文字。  
這是因為位元組在打印時，就好像它們是用 ASCII 編碼的字符一樣。  
（ASCII 是一種較舊的字符編碼，除了英語之外，實際上不適用於書寫任何其他語言。）在這裡，  
你可以看到我們的歐元符號被替換成了一些看起來像 "\xe2\x82\xac" 的文字亂碼，  
當它被當作 ASCII 字符串打印時就是這樣。

In [4]:
# take a look at what the bytes look like
after

b'This is the euro symbol: \xe2\x82\xac'

當我們使用正確的編碼將我們的位元組再次轉換回字符串時，  
我們可以看到我們的文本都正確無誤地顯示出來，這很棒！:)

In [5]:
# convert it back to utf-8
print(after.decode("utf-8"))

This is the euro symbol: €


然而，當我們嘗試使用不同的編碼將我們的位元組映射成字符串時，我們得到了一個錯誤。  
這是因為我們試圖使用的編碼不知道該如何處理我們試圖傳遞給它的位元組。  
你需要告訴 Python 位元組字符串實際上應該使用的編碼。

**你可以將不同的編碼想像成錄製音樂的不同方式。你可以在 CD、卡帶或 8 軌帶上錄製相同的音樂。  
雖然音樂聽起來或多或少是相同的，但你需要使用正確的設備來從每種錄音格式播放音樂。  
正確的解碼器就像是卡帶播放器或 CD 播放器。如果你嘗試在 CD 播放器中播放卡帶，它就不會工作。**

In [6]:
# try to decode our bytes with the ascii encoding
print(after.decode("ascii"))

UnicodeDecodeError: 'ascii' codec can't decode byte 0xe2 in position 25: ordinal not in range(128)

如果我們嘗試使用錯誤的編碼從字符串映射到位元組，我們也會遇到問題。  
正如我之前所說，Python 3 中的字符串預設是 UTF-8，  
所以如果我們嘗試將它們當作另一種編碼處理，我們將會造成問題。

例如，如果我們嘗試使用 `encode()` 將字符串轉換為 ASCII 的位元組，我們可以要求位元組是假設文本為 ASCII 時的樣子。  
然而，由於我們的文本不是 ASCII，會有一些字符它無法處理。我們可以自動替換 ASCII 無法處理的字符。  
但是，如果我們這樣做，任何不在 ASCII 中的字符都將被替換為未知字符。然後，當我們將位元組重新轉換為字符串時，  
該字符將被替換為未知字符。這樣做的危險之處在於無法確定它原本應該是什麼字符。  
這意味著我們可能剛剛使我們的數據變得無法使用！

In [7]:
# start with a string
before = "This is the euro symbol: €"

# encode it to a different encoding, replacing characters that raise errors
after = before.encode("ascii", errors = "replace")

# convert it back to utf-8
print(after.decode("ascii"))

# We've lost the original underlying byte string! It's been 
# replaced with the underlying byte string for the unknown character :(

This is the euro symbol: ?


這很糟糕，我們要避免這樣做！最好是盡快將我們所有的文本轉換為 UTF-8，並保持該編碼。  
將非 UTF-8 輸入轉換為 UTF-8 的最佳時機是在讀取文件時，我們接下來將會談到這個話題。

## 2. Reading in files with encoding problems

你遇到的大多數文件可能都是使用 UTF-8 編碼的。  
這是 Python 預設期望的，所以大多數時候你不會遇到問題。  
然而，有時候你會遇到像這樣的錯誤：

In [8]:
# try to read in a file not in UTF-8
kickstarter_2016 = pd.read_csv("ks-projects-201612.csv")

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x99 in position 7955: invalid start byte

請注意，當我們嘗試將 UTF-8 位元組解碼為 ASCII 時，我們得到的 `UnicodeDecodeError` 與此時相同！  
這告訴我們這個文件實際上不是 UTF-8。雖然我們不知道它實際上是什麼編碼。  
一種弄清楚的方法是嘗試測試一系列不同的字符編碼，看看是否有任何一種能夠工作。  
不過，更好的方法是使用 charset_normalizer 模組嘗試自動猜測正確的編碼。  
這並不保證百分之百正確，但通常比隨便猜測要快。

我打算只查看這個文件的前一萬個位元組。這通常足以對編碼作出一個良好的猜測，且比查看整個文件要快得多。  
（尤其是對於大文件，這可能非常慢。）只查看文件的第一部分的另一個原因是，我們可以通過查看錯誤訊息看到第一個問題是第 11 個字符。  
因此，我們可能只需要查看文件的一小部分就能弄清楚發生了什麼。

In [9]:
# look at the first ten thousand bytes to guess the character encoding
with open("ks-projects-201801.csv", 'rb') as rawdata:
    result = charset_normalizer.detect(rawdata.read(10000))

# check what the character encoding might be
print(result)

{'encoding': 'utf-8', 'language': 'English', 'confidence': 1.0}


因此，charset_normalizer 有 73% 的把握認為正確的編碼是「Windows-1252」。  
讓我們看看這是否正確：

In [10]:
# read in the file with the encoding detected by charset_normalizer
kickstarter_2016 = pd.read_csv("ks-projects-201612.csv", encoding='Windows-1252')

# look at the first few lines
kickstarter_2016.head()

C:\Users\Ling-Hao Lin\AppData\Local\Temp\ipykernel_22936\2942258619.py:2: DtypeWarning: Columns (13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  kickstarter_2016 = pd.read_csv("ks-projects-201612.csv", encoding='Windows-1252')


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09 11:36:00,1000,2015-08-11 12:12:28,0,failed,0,GB,0,NaN,NaN,NaN,NaN
1,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26 00:20:50,45000,2013-01-12 00:20:50,220,failed,3,US,220,NaN,NaN,NaN,NaN
2,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16 04:24:11,5000,2012-03-17 03:24:11,1,failed,1,US,1,NaN,NaN,NaN,NaN
3,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29 01:00:00,19500,2015-07-04 08:35:03,1283,canceled,14,US,1283,NaN,NaN,NaN,NaN
4,1000014025,Monarch Espresso Bar,Restaurants,Food,USD,2016-04-01 13:38:27,50000,2016-02-26 13:38:27,52375,successful,224,US,52375,NaN,NaN,NaN,NaN


是的，看起來 charset_normalizer 是對的！  
文件沒有問題地讀入（雖然我們得到了一個關於資料類型的警告），  
當我們查看前幾行時，似乎一切正常。

**如果 charset_normalizer 猜測的編碼不正確怎麼辦？**  
由於 charset_normalizer 基本上只是一個精緻的猜測器，有時它會猜錯編碼。  
你可以嘗試的一件事是查看更多或更少的文件內容，  
看看是否能得到不同的結果，然後再嘗試那個結果。

## 3. Saving your files with UTF-8 encoding

最後，當你費盡心思將你的文件轉換為 UTF-8 之後，你可能會想保持這種方式。  
最簡單的方法是將你的文件以 UTF-8 編碼保存。好消息是，由於 UTF-8 是 Python 中的標準編碼，  
當你保存一個文件時，它會默認以 UTF-8 的形式保存：

In [11]:
# save our file (will be saved as UTF-8 by default!)
# kickstarter_2016.to_csv("ks-projects-201801-utf8.csv")